# import

In [1]:
import rdkit
from rdkit import Chem
from rdkit.Chem import AllChem
import pandas as pd
from pandas import DataFrame
from os import sep
import numpy as np
import re
from sklearn.preprocessing import OneHotEncoder
import csv
from morfeus import BuriedVolume,Sterimol,XTB,read_xyz
import xtb
from Mol_script import Molecule,get_onestart_path,get_onestart_end_path,get_formed_path

In [5]:
import xtb
xtb.__version__

'20.2'

# function

In [2]:
def sta_smi(smi):
    return Chem.MolToSmiles(Chem.MolFromSmiles(smi))
def sta_list_smi(list_smi):
    return list(map(sta_smi,list_smi))
#描述符归一化
def f_des_std(des,des_name):
    sta_des = []
    sta_des_name = []
    for i in range(np.shape(des)[1]):
        tmp_des = des[:,i]
        if tmp_des.max()!=tmp_des.min():
            tmp_des = (tmp_des-tmp_des.min())/(tmp_des.max()-tmp_des.min())
            sta_des.append(tmp_des)
            sta_des_name.append(des_name[i])
    sta_des = np.transpose(sta_des)
    return sta_des,sta_des_name

# data

## data input

In [3]:
df = pd.read_csv('dataset'+sep+'danmark.csv')

Catalyst = df['Catalyst'].to_list()
Imine = df['Imine'].to_list()
Thiol = df['Thiol'].to_list()
Output = df['Output'].to_list()

df

,Catalyst,Imine,Thiol,Output
0,O=P1(O)OC2=C(C3=CC=CC=C3)C=C4C(C=CC=C4)=C2C5=C...,O=C(C1=CC=CC=C1)/N=C/C2=CC=C(C(F)(F)F)C=C2,CCS,0.501759
1,O=P1(O)OC2=C(C3=C(F)C=C(OC)C=C3F)C=C4C(C=CC=C4...,O=C(C1=CC=CC=C1)/N=C/C2=CC=C(C(F)(F)F)C=C2,CCS,1.074991
2,O=P1(O)OC2=C(C3=C(C)C=C(C)C=C3C)C=C4C(C=CC=C4)...,O=C(C1=CC=CC=C1)/N=C/C2=CC=C(C(F)(F)F)C=C2,CCS,1.179891
3,O=P1(O)OC2=C(C3=CC(C)=C(OC(C)C)C(C)=C3)C=C4C(C...,O=C(C1=CC=CC=C1)/N=C/C2=CC=C(C(F)(F)F)C=C2,CCS,0.446374
4,O=P1(O)OC2=C(C3=CC=C(S(F)(F)(F)(F)F)C=C3)C=C4C...,O=C(C1=CC=CC=C1)/N=C/C2=CC=C(C(F)(F)F)C=C2,CCS,0.530235
...,...,...,...,...
1070,O=P1(O)OC2=C(C3=CC(C(F)(F)F)=CC(C(F)(F)F)=C3)C...,O=C(C1=CC=CC=C1)/N=C/C2=CC=CC3=C2C=CC=C3,SC1CCCCC1,0.682601
1071,O=P1(O)OC2=C(C3=C(C(C)C)C=C(C(C)C)C=C3C(C)C)C=...,O=C(C1=CC=CC=C1)/N=C/C2=CC=CC3=C2C=CC=C3,SC1CCCCC1,1.743658
1072,O=P1(O)OC2=C(C3=C(C4CCCCC4)C=C(C5CCCCC5)C=C3C6...,O=C(C1=CC=CC=C1)/N=C/C2=CC=CC3=C2C=CC=C3,SC1CCCCC1,1.743658
1073,O=P1(O)OC2=C(C3=C(C(C)C)C=C(C4=CC=C(C(C)(C)C)C...,O=C(C1=CC=CC=C1)/N=C/C2=CC=CC3=C2C=CC=C3,SC1CCCCC1,1.446325


# One-Hot descriptor

## Catalyst

### main_ring descriptor

In [5]:
des_cat_main_ring = []
for index,tmp_smi in enumerate(Catalyst):
    tmp_smi = Chem.MolToSmiles(Chem.MolFromSmiles(tmp_smi),kekuleSmiles=False,
                              isomericSmiles=False)
    tmp_mol = Chem.MolFromSmiles(tmp_smi)
    smi_nap = sta_smi('O=P1(OC2=CC=C3C(C=CC=C3)=C2C4=C(C=CC5=C4C=CC=C5)O1)O')
    patt_nap = Chem.MolFromSmarts(smi_nap)
    smi_ben = sta_smi('O=P1(OC2=CC=CC=C2C3=C(C=CC=C3)O1)O')
    patt_ben = Chem.MolFromSmarts(smi_ben)
    if tmp_mol.HasSubstructMatch(patt_nap):
        des_cat_main_ring.append([1,0])
    else:
        des_cat_main_ring.append([0,1])
des_cat_main_ring = np.array(des_cat_main_ring)

In [6]:
des_cat_main_ring_name = ['nap','ben']

### sub_gro descriptor

In [7]:
cat_sub_gro = []
for index,tmp_smi in enumerate(Catalyst):
    tmp_smi = Chem.MolToSmiles(Chem.MolFromSmiles(tmp_smi),kekuleSmiles=False,
                              isomericSmiles=False)
    tmp_mol = Chem.MolFromSmiles(tmp_smi)
    smi_nap = sta_smi('O=P1(OC2=CC=C3C(C=CC=C3)=C2C4=C(C=CC5=C4C=CC=C5)O1)O')
    patt_nap = Chem.MolFromSmarts(smi_nap)
    smi_ben = sta_smi('O=P1(OC2=CC=CC=C2C3=C(C=CC=C3)O1)O')
    patt_ben = Chem.MolFromSmarts(smi_ben)
    if tmp_mol.HasSubstructMatch(patt_nap):
        tmp_Mol = Molecule(tmp_smi,smi_nap)
    else:
        tmp_Mol = Molecule(tmp_smi,smi_ben)
    cat_sub_gro.extend(list(tmp_Mol.dic_sub_gro.keys()))
cat_sub_gro = list(np.unique(cat_sub_gro))
cat_sub_gro_arr = np.array(cat_sub_gro).reshape(-1, 1)
encoder_cat_sub_gro = OneHotEncoder()
encoder_cat_sub_gro.fit(cat_sub_gro_arr)
dic_cat_sub_gro = {}
for smi in cat_sub_gro:
    dic_cat_sub_gro[smi] = list(map(int,encoder_cat_sub_gro.transform(
        [[smi]]).toarray()[0]))

In [8]:
cat_path = []
for index,tmp_smi in enumerate(Catalyst):
    tmp_smi = Chem.MolToSmiles(Chem.MolFromSmiles(tmp_smi),kekuleSmiles=False,
                              isomericSmiles=False)
    tmp_mol = Chem.MolFromSmiles(tmp_smi)
    smi_nap = sta_smi('O=P1(OC2=CC=C3C(C=CC=C3)=C2C4=C(C=CC5=C4C=CC=C5)O1)O')
    patt_nap = Chem.MolFromSmarts(smi_nap)
    smi_ben = sta_smi('O=P1(OC2=CC=CC=C2C3=C(C=CC=C3)O1)O')
    patt_ben = Chem.MolFromSmarts(smi_ben)
    if tmp_mol.HasSubstructMatch(patt_nap):
        tmp_Mol = Molecule(tmp_smi,smi_nap)
    else:
        tmp_Mol = Molecule(tmp_smi,smi_ben)
    smi_P = sta_smi('O=P(O)(O)O')
    patt_P = Chem.MolFromSmiles(smi_P)
    P_matches = list(tmp_mol.GetSubstructMatches(patt_P)[0])
    formed_P_matches = []
    for i in P_matches:
        formed_P_matches.append(tmp_Mol.m2a[i])
    for i in formed_P_matches:
        if sum(tmp_Mol.M_nei[i])==4:
            tmp_P_site = i
    tmp_path = get_onestart_path(tmp_P_site,tmp_Mol.M_nei,tmp_Mol.dic_sub_gro)
    for j in tmp_path:
        tmp_formed_path = get_formed_path(j,tmp_Mol.atom_feature)
        for k in tmp_formed_path:
            if k not in cat_path:
                cat_path.append(k)

In [18]:
cat_pathx = []
for index,tmp_smi in enumerate(Catalyst):
    tmp_smi = Chem.MolToSmiles(Chem.MolFromSmiles(tmp_smi),kekuleSmiles=False,
                              isomericSmiles=False)
    tmp_mol = Chem.MolFromSmiles(tmp_smi)
    smi_nap = sta_smi('O=P1(OC2=CC=C3C(C=CC=C3)=C2C4=C(C=CC5=C4C=CC=C5)O1)O')
    patt_nap = Chem.MolFromSmarts(smi_nap)
    smi_ben = sta_smi('O=P1(OC2=CC=CC=C2C3=C(C=CC=C3)O1)O')
    patt_ben = Chem.MolFromSmarts(smi_ben)
    if tmp_mol.HasSubstructMatch(patt_nap):
        tmp_Mol = Molecule(tmp_smi,smi_nap)
    else:
        tmp_Mol = Molecule(tmp_smi,smi_ben)
    smi_P = sta_smi('O=P(O)(O)O')
    patt_P = Chem.MolFromSmiles(smi_P)
    P_matches = list(tmp_mol.GetSubstructMatches(patt_P)[0])
    formed_P_matches = []
    for i in P_matches:
        formed_P_matches.append(tmp_Mol.m2a[i])
    for i in formed_P_matches:
        if sum(tmp_Mol.M_nei[i])==4:
            tmp_P_site = i
    tmp_path = get_onestart_path(tmp_P_site,tmp_Mol.M_nei,tmp_Mol.dic_sub_gro)
    cat_pathx.append(tmp_path)

In [9]:
des_cat_sub_gro = []
for index,tmp_smi in enumerate(Catalyst):
    tmp_des = []
    tmp_smi = Chem.MolToSmiles(Chem.MolFromSmiles(tmp_smi),kekuleSmiles=False,
                              isomericSmiles=False)
    tmp_mol = Chem.MolFromSmiles(tmp_smi)
    smi_nap = sta_smi('O=P1(OC2=CC=C3C(C=CC=C3)=C2C4=C(C=CC5=C4C=CC=C5)O1)O')
    patt_nap = Chem.MolFromSmarts(smi_nap)
    smi_ben = sta_smi('O=P1(OC2=CC=CC=C2C3=C(C=CC=C3)O1)O')
    patt_ben = Chem.MolFromSmarts(smi_ben)
    if tmp_mol.HasSubstructMatch(patt_nap):
        tmp_Mol = Molecule(tmp_smi,smi_nap)
    else:
        tmp_Mol = Molecule(tmp_smi,smi_ben)
    smi_P = sta_smi('O=P(O)(O)O')
    patt_P = Chem.MolFromSmiles(smi_P)
    P_matches = list(tmp_mol.GetSubstructMatches(patt_P)[0])
    formed_P_matches = []
    for i in P_matches:
        formed_P_matches.append(tmp_Mol.m2a[i])
    for i in formed_P_matches:
        if sum(tmp_Mol.M_nei[i])==4:
            tmp_P_site = i
    tmp_list_sub_gro = list(tmp_Mol.dic_sub_gro.keys())
    for tmp_path in cat_path:
        tmp_path_des = [0]*len(cat_sub_gro)
        for tmp_sub_gro in tmp_list_sub_gro:
            for tmp_sub_gro_site in tmp_Mol.dic_sub_gro[tmp_sub_gro]:
                tmp_formed_sub_gro_path = []
                for j in tmp_sub_gro_site:
                    tmp_sub_gro_path = get_onestart_end_path(
                        tmp_P_site,tmp_Mol.M_nei,j)
                    tmp_formed_sub_gro_path.extend(get_formed_path(
                        tmp_sub_gro_path,tmp_Mol.atom_feature))
                for sub_path in tmp_formed_sub_gro_path:
                    if sub_path==tmp_path:
                        tmp_path_des = [a+b for a,b in zip(tmp_path_des,
                                                dic_cat_sub_gro[tmp_sub_gro])]
        tmp_des.extend(tmp_path_des)
    des_cat_sub_gro.append(tmp_des)
des_cat_sub_gro = np.array(des_cat_sub_gro)

In [10]:
des_cat_sub_gro_name = []
for i in range(len(cat_path)):
    for j in range(len(cat_sub_gro)):
        des_cat_sub_gro_name.append('cat_path-'+str(i)+\
                                           '-'+cat_sub_gro[j])

### path descriptor

In [11]:
cat_path_name = [i for i in range(len(cat_path))]
cat_path_name_arr = np.array(cat_path_name).reshape(-1, 1)
encoder_cat_path_name = OneHotEncoder()
encoder_cat_path_name.fit(cat_path_name_arr)
dic_cat_path = {}
for i,name in enumerate(cat_path_name):
    dic_cat_path[name] = list(map(int,encoder_cat_path_name.transform(
        [[name]]).toarray()[0]))+[len(cat_path[i])]+list(np.sum(
        cat_path[i],axis=0))

In [16]:
des_cat_path = []
for index,tmp_smi in enumerate(Catalyst):
    tmp_des = []
    tmp_smi = Chem.MolToSmiles(Chem.MolFromSmiles(tmp_smi),kekuleSmiles=False,
                              isomericSmiles=False)
    tmp_mol = Chem.MolFromSmiles(tmp_smi)
    smi_nap = sta_smi('O=P1(OC2=CC=C3C(C=CC=C3)=C2C4=C(C=CC5=C4C=CC=C5)O1)O')
    patt_nap = Chem.MolFromSmarts(smi_nap)
    smi_ben = sta_smi('O=P1(OC2=CC=CC=C2C3=C(C=CC=C3)O1)O')
    patt_ben = Chem.MolFromSmarts(smi_ben)
    if tmp_mol.HasSubstructMatch(patt_nap):
        tmp_Mol = Molecule(tmp_smi,smi_nap)
    else:
        tmp_Mol = Molecule(tmp_smi,smi_ben)
    smi_P = sta_smi('O=P(O)(O)O')
    patt_P = Chem.MolFromSmiles(smi_P)
    P_matches = list(tmp_mol.GetSubstructMatches(patt_P)[0])
    formed_P_matches = []
    for i in P_matches:
        formed_P_matches.append(tmp_Mol.m2a[i])
    for i in formed_P_matches:
        if sum(tmp_Mol.M_nei[i])==4:
            tmp_P_site = i
    tmp_list_sub_gro = list(tmp_Mol.dic_sub_gro.keys())
    for tmp_sub_gro0 in cat_sub_gro:
        tmp_sub_gro_des = [0]*(len(cat_path_name)+9)
        for tmp_sub_gro in tmp_list_sub_gro:
            if tmp_sub_gro0==tmp_sub_gro:
                for tmp_sub_gro_site in tmp_Mol.dic_sub_gro[tmp_sub_gro]:
                    tmp_formed_sub_gro_path = []
                    for j in tmp_sub_gro_site:
                        tmp_sub_gro_path = get_onestart_end_path(
                            tmp_P_site,tmp_Mol.M_nei,j)
                        tmp_formed_sub_gro_path.extend(get_formed_path(
                            tmp_sub_gro_path,tmp_Mol.atom_feature))
                for pathx in tmp_formed_sub_gro_path:
                    path_index = cat_path.index(pathx)
                    tmp_sub_gro_des = [a+b for a,b in zip(tmp_sub_gro_des,
                                            dic_cat_path[path_index])]
        tmp_des.extend(tmp_sub_gro_des)
    des_cat_path.append(tmp_des)
des_cat_path = np.array(des_cat_path)

In [17]:
des_cat_path_name = []
cat_path_des_name = ['cat_path-'+str(i) for i in range(len(cat_path))]+\
['C_num','O_num','P_num','sp3','sp2','sp','H_num','Degree_num','aro_num']
for i in range(len(cat_sub_gro)):
    for j in range(len(cat_path_des_name)):
        des_cat_path_name.append(cat_sub_gro[i]+'-'+cat_path_des_name[j])

## Imine

In [18]:
sta_Imine = list(np.unique(sta_list_smi(Imine)))
sta_Imine_arr = np.array(sta_Imine).reshape(-1,1)
encoder_Imine = OneHotEncoder()
encoder_Imine.fit(sta_Imine_arr)
des_Imine = encoder_Imine.transform(np.array(sta_list_smi(
    Imine)).reshape(-1,1)).toarray()
des_Imine_name = sta_Imine[:]

## Thiol

In [19]:
sta_Thiol = list(np.unique(sta_list_smi(Thiol)))
sta_Thiol_arr = np.array(sta_Thiol).reshape(-1,1)
encoder_Thiol = OneHotEncoder()
encoder_Thiol.fit(sta_Thiol_arr)
des_Thiol = encoder_Thiol.transform(np.array(sta_list_smi(
    Thiol)).reshape(-1,1)).toarray()
des_Thiol_name = sta_Thiol[:]

## all descriptor

In [20]:
des = np.concatenate((des_cat_main_ring,des_cat_sub_gro,des_cat_path,des_Imine,
                      des_Thiol),axis=1)
des_name = des_cat_main_ring_name+des_cat_sub_gro_name+des_cat_path_name+\
    des_Imine_name+des_Thiol_name
#描述符正规化
sta_des,sta_des_name = f_des_std(des,des_name)

In [27]:
#存储path_OH描述符
with open('descriptor'+sep+'des_danmark_path_onehot.csv','w',newline='')as f:
    csv_writer = csv.writer(f)
    csv_writer.writerow(sta_des_name)
    for index,tmp_des in enumerate(sta_des):
        csv_writer.writerow(list(tmp_des))

# ste_ele descriptor

## Catalyst

### main_ring descriptor

In [22]:
des_cat_main_ring_OH = []
for index,tmp_smi in enumerate(Catalyst):
    tmp_smi = Chem.MolToSmiles(Chem.MolFromSmiles(tmp_smi),kekuleSmiles=False,
                              isomericSmiles=False)
    tmp_mol = Chem.MolFromSmiles(tmp_smi)
    smi_nap = sta_smi('O=P1(OC2=CC=C3C(C=CC=C3)=C2C4=C(C=CC5=C4C=CC=C5)O1)O')
    patt_nap = Chem.MolFromSmarts(smi_nap)
    smi_ben = sta_smi('O=P1(OC2=CC=CC=C2C3=C(C=CC=C3)O1)O')
    patt_ben = Chem.MolFromSmarts(smi_ben)
    if tmp_mol.HasSubstructMatch(patt_nap):
        des_cat_main_ring_OH.append([1,0])
    else:
        des_cat_main_ring_OH.append([0,1])
des_cat_main_ring_OH = np.array(des_cat_main_ring_OH)
des_cat_main_ring_OH_name = ['nap','ben']

### sub_gro descriptor

In [23]:
cat_sub_gro = []
for index,tmp_smi in enumerate(Catalyst):
    tmp_smi = Chem.MolToSmiles(Chem.MolFromSmiles(tmp_smi),kekuleSmiles=False,
                              isomericSmiles=False)
    tmp_mol = Chem.MolFromSmiles(tmp_smi)
    smi_nap = sta_smi('O=P1(OC2=CC=C3C(C=CC=C3)=C2C4=C(C=CC5=C4C=CC=C5)O1)O')
    patt_nap = Chem.MolFromSmarts(smi_nap)
    smi_ben = sta_smi('O=P1(OC2=CC=CC=C2C3=C(C=CC=C3)O1)O')
    patt_ben = Chem.MolFromSmarts(smi_ben)
    if tmp_mol.HasSubstructMatch(patt_nap):
        tmp_Mol = Molecule(tmp_smi,smi_nap)
    else:
        tmp_Mol = Molecule(tmp_smi,smi_ben)
    cat_sub_gro.extend(list(tmp_Mol.dic_sub_gro.keys()))
cat_sub_gro = list(np.unique(cat_sub_gro))

In [24]:
#dic_des_steric
formed_cat_sub_gro_steric = cat_sub_gro[:]
formed_cat_sub_gro_steric = [s.replace('*','I') for s in formed_cat_sub_gro_steric]
dic_cat_sub_gro_steric = {}
flag = True
for i,tmp_smi in enumerate(formed_cat_sub_gro_steric):
    tmp_mol = Chem.MolFromSmiles(tmp_smi)
    tmp_mol = AllChem.AddHs(tmp_mol)
    AllChem.EmbedMolecule(tmp_mol)
    AllChem.MMFFOptimizeMoleculeConfs(tmp_mol)
    tmp_xyz = Chem.MolToXYZBlock(tmp_mol)
    tmp_xyz = list(tmp_xyz.replace(str(tmp_mol.GetNumAtoms())+'\n\n',''
                                      )[:-1].split('\n'))
    elements = [s.split()[0] for s in tmp_xyz]
    coordinates = np.array([list(map(float,s.split()[1:])) for s in tmp_xyz])
    ste = Sterimol(elements,coordinates,1,2,radii = [i+1 for i in range(10)])
    tmp_dic_bv_ste = [ste.L_value,ste.B_1_value,ste.B_5_value]
    if flag:
        ste_bv_name = ['steL','steB1','steB5']
    for j in range(10):
        bv = BuriedVolume(elements,coordinates,1,z_axis_atoms=2,radius=j+1)
        tmp_dic_bv_ste.append(bv.fraction_buried_volume)
        if flag:
            ste_bv_name.extend(['bv_r'+str(j+1)])
    flag = False
    dic_cat_sub_gro_steric[cat_sub_gro[i]] = tmp_dic_bv_ste

[15:33:55] UFFTYPER: Unrecognized atom type: S_6+6 (5)


In [25]:
#dic_des_ele
formed_cat_sub_gro_ele = cat_sub_gro[:]
formed_cat_sub_gro_ele = [s.replace('*','') for s in formed_cat_sub_gro_ele]
formed_cat_sub_gro_ele = [s.replace('[Si]','[H][Si]')\
                          for s in formed_cat_sub_gro_ele]
dic_cat_sub_gro_ele = {}
flag = True
for i,tmp_smi in enumerate(formed_cat_sub_gro_ele):
    tmp_mol = Chem.MolFromSmiles(tmp_smi)
    tmp_mol = AllChem.AddHs(tmp_mol)
    AllChem.EmbedMolecule(tmp_mol)
    AllChem.MMFFOptimizeMoleculeConfs(tmp_mol)
    tmp_xyz = Chem.MolToXYZBlock(tmp_mol)
    tmp_xyz = list(tmp_xyz.replace(str(tmp_mol.GetNumAtoms())+'\n\n',''
                                      )[:-1].split('\n'))
    elements = [s.split()[0] for s in tmp_xyz]
    coordinates = np.array([list(map(float,s.split()[1:])) for s in tmp_xyz])
    tmp_xtb = XTB(elements, coordinates)
    tmp_charge = tmp_xtb.get_charges()[1]
    tmp_dipole = list(tmp_xtb.get_dipole())
    tmp_ea = tmp_xtb.get_ea()
    tmp_fuki = []
    list_variety0 = ['dual','electrophilicity','local_electrophilicity',
                    'local_nucleophilicity','nucleophilicity','radical']
    for variety in list_variety0:
        tmp_fuki.append(tmp_xtb.get_fukui(variety)[1])
    tmp_glo = []
    list_variety1 = ['electrofugality','electrophilicity','nucleofugality',
                     'nucleophilicity']
    for variety in list_variety1:
        tmp_glo.append(tmp_xtb.get_global_descriptor(variety))
    tmp_homo = tmp_xtb.get_homo()
    tmp_ip = tmp_xtb.get_ip()
    tmp_lumo = tmp_xtb.get_lumo()
    tmp_ele = [tmp_charge]+tmp_dipole+[tmp_ea]+tmp_fuki+tmp_glo+[tmp_homo,
              tmp_ip,tmp_lumo]
    if flag:
        ele_name = ['Charge0','dipole1','dipole2','dipole3','ea']+[
            'fukui_'+i+'0' for i in list_variety0]+[
            'global_'+i for i in list_variety1]+['homo','ip','lumo']
        flag = False
    dic_cat_sub_gro_ele[cat_sub_gro[i]] = tmp_ele

[15:35:28] UFFTYPER: Unrecognized atom type: S_6+6 (4)


In [26]:
dic_cat_sub_gro_ste_ele = {}
for i,tmp_smi in enumerate(cat_sub_gro):
    dic_cat_sub_gro_ste_ele[tmp_smi] = dic_cat_sub_gro_steric[tmp_smi]+\
    dic_cat_sub_gro_ele[tmp_smi]
ste_ele_name = ste_bv_name+ele_name

In [27]:
cat_path = []
for index,tmp_smi in enumerate(Catalyst):
    tmp_smi = Chem.MolToSmiles(Chem.MolFromSmiles(tmp_smi),kekuleSmiles=False,
                              isomericSmiles=False)
    tmp_mol = Chem.MolFromSmiles(tmp_smi)
    smi_nap = sta_smi('O=P1(OC2=CC=C3C(C=CC=C3)=C2C4=C(C=CC5=C4C=CC=C5)O1)O')
    patt_nap = Chem.MolFromSmarts(smi_nap)
    smi_ben = sta_smi('O=P1(OC2=CC=CC=C2C3=C(C=CC=C3)O1)O')
    patt_ben = Chem.MolFromSmarts(smi_ben)
    if tmp_mol.HasSubstructMatch(patt_nap):
        tmp_Mol = Molecule(tmp_smi,smi_nap)
    else:
        tmp_Mol = Molecule(tmp_smi,smi_ben)
    smi_P = sta_smi('O=P(O)(O)O')
    patt_P = Chem.MolFromSmiles(smi_P)
    P_matches = list(tmp_mol.GetSubstructMatches(patt_P)[0])
    formed_P_matches = []
    for i in P_matches:
        formed_P_matches.append(tmp_Mol.m2a[i])
    for i in formed_P_matches:
        if sum(tmp_Mol.M_nei[i])==4:
            tmp_P_site = i
    tmp_path = get_onestart_path(tmp_P_site,tmp_Mol.M_nei,tmp_Mol.dic_sub_gro)
    for j in tmp_path:
        tmp_formed_path = get_formed_path(j,tmp_Mol.atom_feature)
        for k in tmp_formed_path:
            if k not in cat_path:
                cat_path.append(k)

#### ste_ele

In [28]:
des_cat_sub_gro_ste_ele = []
for index,tmp_smi in enumerate(Catalyst):
    tmp_des = []
    tmp_smi = Chem.MolToSmiles(Chem.MolFromSmiles(tmp_smi),kekuleSmiles=False,
                              isomericSmiles=False)
    tmp_mol = Chem.MolFromSmiles(tmp_smi)
    smi_nap = sta_smi('O=P1(OC2=CC=C3C(C=CC=C3)=C2C4=C(C=CC5=C4C=CC=C5)O1)O')
    patt_nap = Chem.MolFromSmarts(smi_nap)
    smi_ben = sta_smi('O=P1(OC2=CC=CC=C2C3=C(C=CC=C3)O1)O')
    patt_ben = Chem.MolFromSmarts(smi_ben)
    if tmp_mol.HasSubstructMatch(patt_nap):
        tmp_Mol = Molecule(tmp_smi,smi_nap)
    else:
        tmp_Mol = Molecule(tmp_smi,smi_ben)
    smi_P = sta_smi('O=P(O)(O)O')
    patt_P = Chem.MolFromSmiles(smi_P)
    P_matches = list(tmp_mol.GetSubstructMatches(patt_P)[0])
    formed_P_matches = []
    for i in P_matches:
        formed_P_matches.append(tmp_Mol.m2a[i])
    for i in formed_P_matches:
        if sum(tmp_Mol.M_nei[i])==4:
            tmp_P_site = i
    tmp_list_sub_gro = list(tmp_Mol.dic_sub_gro.keys())
    for tmp_path in cat_path:
        tmp_path_des = [0]*len(ste_ele_name)
        for tmp_sub_gro in tmp_list_sub_gro:
            for tmp_sub_gro_site in tmp_Mol.dic_sub_gro[tmp_sub_gro]:
                tmp_formed_sub_gro_path = []
                for j in tmp_sub_gro_site:
                    tmp_sub_gro_path = get_onestart_end_path(
                        tmp_P_site,tmp_Mol.M_nei,j)
                    tmp_formed_sub_gro_path.extend(get_formed_path(
                        tmp_sub_gro_path,tmp_Mol.atom_feature))
                for sub_path in tmp_formed_sub_gro_path:
                    if sub_path==tmp_path:
                        tmp_path_des = [a+b for a,b in zip(tmp_path_des,
                                        dic_cat_sub_gro_ste_ele[tmp_sub_gro])]
        tmp_des.extend(tmp_path_des)
    des_cat_sub_gro_ste_ele.append(tmp_des)
des_cat_sub_gro_ste_ele = np.array(des_cat_sub_gro_ste_ele)

In [29]:
des_cat_sub_gro_ste_ele_name = []
for i in range(len(cat_path)):
    for j in range(len(ste_ele_name)):
        des_cat_sub_gro_ste_ele_name.append('cat_path-'+str(i)+\
                                           '-'+ste_ele_name[j])

### path descriptor

In [30]:
cat_path_name = [i for i in range(len(cat_path))]
cat_path_name_arr = np.array(cat_path_name).reshape(-1, 1)
encoder_cat_path_name = OneHotEncoder()
encoder_cat_path_name.fit(cat_path_name_arr)
dic_cat_path = {}
for i,name in enumerate(cat_path_name):
    dic_cat_path[name] = list(map(int,encoder_cat_path_name.transform(
        [[name]]).toarray()[0]))+[len(cat_path[i])]+list(np.sum(
        cat_path[i],axis=0))

In [31]:
des_cat_path = []
for index,tmp_smi in enumerate(Catalyst):
    tmp_des = []
    tmp_smi = Chem.MolToSmiles(Chem.MolFromSmiles(tmp_smi),kekuleSmiles=False,
                              isomericSmiles=False)
    tmp_mol = Chem.MolFromSmiles(tmp_smi)
    smi_nap = sta_smi('O=P1(OC2=CC=C3C(C=CC=C3)=C2C4=C(C=CC5=C4C=CC=C5)O1)O')
    patt_nap = Chem.MolFromSmarts(smi_nap)
    smi_ben = sta_smi('O=P1(OC2=CC=CC=C2C3=C(C=CC=C3)O1)O')
    patt_ben = Chem.MolFromSmarts(smi_ben)
    if tmp_mol.HasSubstructMatch(patt_nap):
        tmp_Mol = Molecule(tmp_smi,smi_nap)
    else:
        tmp_Mol = Molecule(tmp_smi,smi_ben)
    smi_P = sta_smi('O=P(O)(O)O')
    patt_P = Chem.MolFromSmiles(smi_P)
    P_matches = list(tmp_mol.GetSubstructMatches(patt_P)[0])
    formed_P_matches = []
    for i in P_matches:
        formed_P_matches.append(tmp_Mol.m2a[i])
    for i in formed_P_matches:
        if sum(tmp_Mol.M_nei[i])==4:
            tmp_P_site = i
    tmp_list_sub_gro = list(tmp_Mol.dic_sub_gro.keys())
    for tmp_sub_gro0 in cat_sub_gro:
        tmp_sub_gro_des = [0]*(len(cat_path_name)+9)
        for tmp_sub_gro in tmp_list_sub_gro:
            if tmp_sub_gro0==tmp_sub_gro:
                for tmp_sub_gro_site in tmp_Mol.dic_sub_gro[tmp_sub_gro]:
                    tmp_formed_sub_gro_path = []
                    for j in tmp_sub_gro_site:
                        tmp_sub_gro_path = get_onestart_end_path(
                            tmp_P_site,tmp_Mol.M_nei,j)
                        tmp_formed_sub_gro_path.extend(get_formed_path(
                            tmp_sub_gro_path,tmp_Mol.atom_feature))
                for pathx in tmp_formed_sub_gro_path:
                    path_index = cat_path.index(pathx)
                    tmp_sub_gro_des = [a+b for a,b in zip(tmp_sub_gro_des,
                                            dic_cat_path[path_index])]
        tmp_des.extend(tmp_sub_gro_des)
    des_cat_path.append(tmp_des)
des_cat_path = np.array(des_cat_path)

In [32]:
des_cat_path_name = []
cat_path_des_name = ['cat_path-'+str(i) for i in range(len(cat_path))]+\
['C_num','O_num','P_num','sp3','sp2','sp','H_num','Degree_num','aro_num']
for i in range(len(cat_sub_gro)):
    for j in range(len(cat_path_des_name)):
        des_cat_path_name.append(cat_sub_gro[i]+'-'+cat_path_des_name[j])

## Imine

In [34]:
sta_Imine = list(np.unique(sta_list_smi(Imine)))
sta_Imine_arr = np.array(sta_Imine).reshape(-1,1)
encoder_Imine = OneHotEncoder()
encoder_Imine.fit(sta_Imine_arr)
des_Imine_OH = encoder_Imine.transform(np.array(sta_list_smi(
    Imine)).reshape(-1,1)).toarray()
des_Imine_OH_name = sta_Imine[:]

## Thiol

In [35]:
sta_Thiol = list(np.unique(sta_list_smi(Thiol)))
sta_Thiol_arr = np.array(sta_Thiol).reshape(-1,1)
encoder_Thiol = OneHotEncoder()
encoder_Thiol.fit(sta_Thiol_arr)
des_Thiol_OH = encoder_Thiol.transform(np.array(sta_list_smi(
    Thiol)).reshape(-1,1)).toarray()
des_Thiol_OH_name = sta_Thiol[:]

## all descriptor

### ste_ele

In [36]:
des_ste_ele = np.concatenate((des_cat_main_ring_OH,des_cat_sub_gro_ste_ele,
                      des_cat_path,des_Imine_OH,des_Thiol_OH),axis=1)
des_ste_ele_name = des_cat_main_ring_OH_name+des_cat_sub_gro_ste_ele_name+\
des_cat_path_name+des_Imine_OH_name+des_Thiol_OH_name
#描述符正规化
sta_des_ste_ele,sta_des_ste_ele_name = f_des_std(des_ste_ele,
                                                 des_ste_ele_name)

In [37]:
np.shape(des_ste_ele),np.shape(sta_des_ste_ele)

((1075, 1693), (1075, 1332))

In [38]:
#存储ste_ele描述符
with open('descriptor'+sep+'des_danmark_ste_ele.csv','w',newline='')as f:
    csv_writer = csv.writer(f)
    csv_writer.writerow(sta_des_ste_ele_name)
    for index,tmp_des in enumerate(sta_des_ste_ele):
        csv_writer.writerow(list(tmp_des))